In [1]:
#importing modules which is used in simple RAG Project.
#below classes we used so user can interact with LLM Models.
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint #HFE class we used to hit user query and take response from it.

#below classes we used for embedding Models
from langchain_openai import OpenAIEmbeddings  #close source model
from langchain_huggingface import HuggingFaceEmbeddings #open source model

import pdfplumber

#want to load document into workingspace using document loaders
from langchain_community.document_loaders import PDFPlumberLoader,TextLoader

#now splitting the document into chunks need splitter class
from langchain.text_splitter import RecursiveCharacterTextSplitter #this split the text document through herirachy way.

#need to store the chunks embedded document to vector store we r using Pinecone.
from pinecone import Pinecone

#if i want to add temporary memeory in simple rag we have to used
from langchain.memory import ConversationBufferMemory


#load the env files
from dotenv import load_dotenv

from pathlib import Path
from typing import Annotated,Optional,List,Literal,TypedDict
from dataclasses import dataclass

load_dotenv()

True

In [2]:
from loggers import logger
from Exception import CustomException
import os,sys

# step:1) creating an object of model class so that user will interact with LLM Model

In [3]:
#groq model
model1 = ChatGroq(
    model="groq/compound-mini",
    temperature=0.1
)

#openai model
model2 = ChatOpenAI(
    model="gpt-3.5-turbo",temperature=0.1
)

#hugging face model.
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",  
    task="text-generation",  
    
)
model3 = ChatHuggingFace(llm=llm)

# initializing the embedding Models

In [4]:
emb_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

# loading the pdf file and extracting the text content from all pages

In [5]:
def get_pdf_loader(pdf_path:Path)->list[str]:
    """
    Extracts text from all pages of a PDF file using pdfplumber and 
    stores it in a single string.
    
    """
    #creating an object of PDFPlumberLoader class
    try:
        
        loader = PDFPlumberLoader(
            file_path=pdf_path,
            extract_images=False
        )
        
        logger.info("PDFPlumberLoader Loader object Created")
        
        #now loading the pdfs in my work space.
        doc_load = loader.load() 
        logger.info(f"Document Loaded {doc_load}")
        
        #now loading the pdf file into work space.
    except Exception as e:
        raise CustomException(e,sys)
    
    return doc_load

In [6]:
text = get_pdf_loader(pdf_path=Path("Medical_book.pdf"))

[2025-09-20 18:12:49,659]-INFO-15-PDFPlumberLoader Loader object Created
[2025-09-20 18:15:12,129]-INFO-19-Document Loaded [Document(metadata={'source': 'Medical_book.pdf', 'file_path': 'Medical_book.pdf', 'page': 0, 'total_pages': 637, 'ModDate': "D:20041218161531-06'00'", 'CreationDate': "D:20041218170002-05'00'", 'Producer': 'PDFlib+PDI 5.0.0 (SunOS)'}, page_content='\n'), Document(metadata={'source': 'Medical_book.pdf', 'file_path': 'Medical_book.pdf', 'page': 1, 'total_pages': 637, 'ModDate': "D:20041218161531-06'00'", 'CreationDate': "D:20041218170002-05'00'", 'Producer': 'PDFlib+PDI 5.0.0 (SunOS)'}, page_content='GALE\nThe\nENCYCLOPEDIA\nM\nEDICINE\nof\nSECOND EDITION\n'), Document(metadata={'source': 'Medical_book.pdf', 'file_path': 'Medical_book.pdf', 'page': 2, 'total_pages': 637, 'ModDate': "D:20041218161531-06'00'", 'CreationDate': "D:20041218170002-05'00'", 'Producer': 'PDFlib+PDI 5.0.0 (SunOS)'}, page_content='G ALE\nThe\nENCYCLOPEDIA\nM\nEDICINE\nof\nSECOND EDITION\nV O 

# splitting document into chunks

In [7]:
def get_chunks_document(doc:List[str]) ->List[str]:
    """
    here we are using document splitting technique
    in which we gonna used RecursiveCharacterTextSplitter this algorithm will split the document based
    on Text Structure following Heirarchy
    """
    try:
        #creating an object of RecursiveCharacterTextSplitter class.
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap = 50
        )
        
        #now splitting the document into chunks.
        chunks = splitter.split_documents(documents=doc)
        
        logger.info("splitting the document done and created chunks successfully")
    except Exception as e:
        raise CustomException(e,sys)
    
    return chunks

In [8]:
chunks = get_chunks_document(doc=text)
print(f"length of chunks: {len(chunks)}")
print(chunks[0])

[2025-09-20 18:15:12,414]-INFO-17-splitting the document done and created chunks successfully
length of chunks: 3016
page_content='GALE
The
ENCYCLOPEDIA
M
EDICINE
of
SECOND EDITION' metadata={'source': 'Medical_book.pdf', 'file_path': 'Medical_book.pdf', 'page': 1, 'total_pages': 637, 'ModDate': "D:20041218161531-06'00'", 'CreationDate': "D:20041218170002-05'00'", 'Producer': 'PDFlib+PDI 5.0.0 (SunOS)'}


## Pinecone Vector Database Setup

In [9]:
from pinecone import Pinecone, ServerlessSpec

#creating an object of Pinecone database class
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)

#checking wheather index exist or not
index_name = "medical-book-index" #Always<-- hyphen instead of underscore
try:
    if index_name not in pc.list_indexes():
        pc.create_index(
            name=index_name,
            vector_type="dense",
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
            tags={"environment": "RAGdevelopment"}
        )
    logger.info("Pinecone Index Created or Already Exists")
except Exception as e:
    if "ALREADY_EXISTS" in str(e):
        logger.info(f"Index '{index_name}' already exists, skipping creation.")
    else:
        raise CustomException(e, sys)


[2025-09-20 18:15:16,593]-INFO-23-Index 'medical-book-index' already exists, skipping creation.


In [10]:
#Target the index
index = pc.Index(index_name)
index

# step:2) now developing Pinecone Database using langchain

In [11]:
#now giving unique indentity to embedding vectors.
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(chunks))]
uuids[:5]

['8e7a3d5f-1d95-49cb-b6c9-1d572caea9c9',
 '3240ddd0-5bb8-48b9-9b22-ef5ccf2f0fb9',
 '2e6b2a85-2999-4996-8d47-22429d37e9f1',
 '241e75bd-c93f-4721-b2da-34b64cfeea3f',
 '77f301ff-9611-4bd8-afff-16247840bfb0']

In [12]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore.from_documents(
    documents=chunks,      # your document chunks
    embedding=emb_model,   # your embedding model
    ids=uuids,             # unique IDs
    index_name=index_name   # important
)

c:\anaconda\envs\hf-gpu\lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


[2025-09-20 18:15:28,615]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-09-20 18:15:52,244]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-09-20 18:16:07,731]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-09-20 18:16:18,740]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


# now performing the similarity search

In [19]:
results = vector_store.similarity_search_with_score(
    query = "what is acne?",k=2
    
)

for doc,score in results:
    print(doc.page_content)

[2025-09-20 18:25:03,056]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
disease specialist,or an endocrinologist,a specialist who
treats diseases of the body’s endocrine (hormones and
glands) system.
Acne has a characteristic appearance and is not diffi-
cult to diagnose. The doctor takes a complete medical
history,including questions about skin care,diet,factors
causing flare-ups,medication use,and prior treatment.
Physical examination includes the face, upper neck,
chest,shoulders,back,and other affected areas. Under
good lighting,the doctor determines what types and how
many blemishes are present,whether they are inflamed,
whether they are deep or superficial,and whether there is
scarring or skin discoloration.
In teenagers, acne is often found on the forehead,
nose,and chin. As people get older,acne tends to appear
towards the outer part of the face. Adult women may
have acne on their chins and around their mouths. The
elderly may develop white

# step:3) converting vectorstore to become retrievers

In [20]:
retriever = vector_store.as_retriever(
    
    search_type="mmr" #mmr means maximum marginal relevancy or we can take consine similarity
    #mmr will give more diversified search output
    ,search_kwargs={"k": 2, "lambda_mult": 0.25} 
    #range lie between 0 to 1 o means behave like similarity search 1 means very diverse out 
    
)
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x00000260C8A43C10>, search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25})

# now combining the retrieve document

In [34]:
def combine_retreieve_doc(retrieve_doc:List[str])->str:
    """
    we will get the retreival document in list of str format 
    now combining all retriever document together 
    return str object
    """
    lst_doc = []
    for content in  retrieve_doc:
        lst_doc.append(content.page_content)
    
    join_doc = "\n\n".join(lst_doc)
    
    return join_doc

In [31]:
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### forming sequential chain

In [37]:
#forming sequential chain that take rag prompt and send to llm to generate answer
from langchain_core.runnables import RunnableSequence,RunnableLambda,RunnablePassthrough,RunnableParallel
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
seq_chain = rag_prompt | model2 | parser 

### now forming parallel chain

In [39]:
parallel_chain = RunnableParallel({
    #taking user query at time invoke we know retriever is runnable based on fetching contxtual document from vector database.
    "context": retriever | RunnableLambda(combine_retreieve_doc), #we will get context document from DB and now joining doc
    'question': RunnablePassthrough()
    
    
})
parallel_chain


{
  context: VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x00000260C8A43C10>, search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25})
           | RunnableLambda(combine_retreieve_doc),
  question: RunnablePassthrough()
}

# combining both chain together

In [40]:
final_chain = parallel_chain | seq_chain
final_chain

{
  context: VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x00000260C8A43C10>, search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25})
           | RunnableLambda(combine_retreieve_doc),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nConte

In [43]:
print("PDF RAG ready. Ask a question")
#final chain is runnable so we can invoke easily.
result = final_chain.invoke(input = "what is acne?")
result

PDF RAG ready. Ask a question
[2025-09-20 20:15:26,348]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-09-20 20:15:30,945]-INFO-1025-HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Acne is a common skin condition characterized by blemishes on the face, neck, chest, shoulders, and back. It is often diagnosed based on physical examination and medical history, including factors like skin care, diet, and prior treatment. Acne can be treated with various antiacne drugs that may have side effects like dry skin, itching, and redness.'